## Lab for Linear Regression

### Linear Algebra in Python/Numpy

In this lab we will use:
- the `numpy` linear algebra package for computations
- the `bokeh` plotting package for graphics

The next cell loads these libraries.

In [1]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

This routine will save typing later.

In [ ]:
def comparison_plot(Y, Yhat):
    '''Plots Predicted vs True values for analysis of regression'''
    comparison_plot = figure(width=300,height=300)
    comparison_plot.xaxis.axis_label='Y'
    comparison_plot.yaxis.axis_label='Yhat'
    comparison_plot.scatter(x=Y,y=Yhat)
    comparison_plot.line(x=[Y.min(),Y.max()],y=[Y.min(),Y.max()])
    return comparison_plot


### Distance to a moving object (simulated data)

We will begin by looking at Figures 1 and 2 in the [Linear Regression](../notes/main.pdf) notes.

In Figure 1 we began with simulated data that represents the distance from an observer to a a moving object. 
We will generate that data.  The observations occur at times $0, 1, 2, \ldots, 9$.  

In [ ]:
# Create a 1-d numpy array containing 0,1,...,9 
x = np.array(range(10))

Let's suppose that the true velocity of the object is $15$ m/s and the initial distance to the object is $150$ m.  Then
the object's position is given by $d=150-15x$ where $x$ is time.  However, when we measure the distance, there is a random error
of between $-10$ and $10$ meters. The function `np.random.uniform` returns uniformly distributed random numbers within a given range.

In [ ]:
y = 150-15*x+np.random.uniform(-10,10,size=x.shape[0])

We will plot the data points $(x,y)$.

In [ ]:
f=figure(width=400,height=400,title='Measured Distance to Moving Object')
f.scatter(x=x,y=y)
f.xaxis.axis_label='time'
f.yaxis.axis_label='distance (m)'
show(f)

The points are roughly on a line, but not exactly, due to the error.  The
data matrix $X$ for this problem consists of two columns, one of which is $0,1,2,\ldots, 9$ and the other
is $1,1,1,1,\ldots$. (See section 1.3 in the notes).

We make this by:
- converting x to a column vector
- creating a column vector of 1's
- concatenating these

In [ ]:
X=np.concatenate([x.reshape(-1,1),np.ones(shape=(x.shape[0],1))],axis=1)

In [ ]:
X

The Y vector is our vector of measurements, except we need to make it a column vector.

In [ ]:
Y = y.reshape(-1,1)

In [ ]:
Y

Now we will use the formulae in equations (7) and (8) from the notes t compute the least squares line.


In [ ]:
D = np.dot(X.transpose(),X)

The matrix $M$ contains the slope and intercept of our least squares line.

In [ ]:
M = np.dot(np.linalg.inv(D),np.dot(X.transpose(), Y))
print('Slope is {}, Intercept is {}'.format(M[0,0],M[1,0]))

The predicted values of $Y$ are given by equation (8).

In [ ]:
Yhat = np.dot(np.dot(np.dot(X,np.linalg.inv(D)),X.transpose()),Y)

Let's add these predicted values to our plot for comparison.  Notice that the green
dots lie along the regression line.

In [ ]:
f.scatter(x=x,y=Yhat[:,0],color='green')
show(f)

We can connect the dots to see the line of best fit.

In [ ]:
f.line(x=x,y=Yhat[:,0],color='green')
show(f)

In [ ]:
Y.shape
Yhat.shape

In [ ]:
show(comparison_plot(Y[:,0], Yhat[:,0]))

### The MPG Data

Figure 2 in the notes shows data relating engine size to mileage for a group of cars.  Your task is to reproduce the graph
show in the figure.

#### Step 1.  Load the data

The mileage data is in a comma separated file (csv) called `../data/auto-mpg/auto-mpg.csv`  The command `np.genfromtxt`
can be used to read in an array like this.  You can examine the file by using the jupyter file browser and double clicking on the file name.  You will see that:
- the first row is a header
- the last column is the type of car, which is a string; numpy can't handle that so it will set them to nan meaning 'not a number'
- mpg is column zero
- displacement is column 2

In [ ]:
data = np.genfromtxt('../data/auto-mpg/auto-mpg.csv',delimiter=',',skip_header=1)
data

Now your job is to complete the code below, following the work we did above, so that you:
- Create  variable $x$ that is just column 2 (displacement) of the data matrix 
- create variable $Y$ that is just column 0 (mpg) of the data matrix 
- Plot $x$ and $Y$ as a scatter plot.
- Create a matrix whose first column is $x$ and whose second column is all $1$.
- Compute the matrix $D=X^{\intercal}X$
- Find $M=D^{-1}X^{\intercal}Y$
- Find $Yhat= XD^{-1}X^{\intercal}Y$
- Plot $Yhat$ and the predicted line.

In [ ]:
#x=data[]
#Y=data[]
#f = figure()
#f.scatter()
#show(f)


In [ ]:
x=data[:,2]
Y=data[:,0]
f=figure()
f.scatter(x=x,y=Y)
show(f)

In [ ]:

#X = np.concatenate([],axis=1)
#D = np.dot(...)
#M = np.dot(...)
#Yhat = ...
#f.scatter(...,color='green')
#f.line(...,color='green')

In [ ]:
X=np.concatenate([x.reshape(-1,1),np.ones((x.shape[0],1))],axis=1)
D = np.dot(X.transpose(),X)
M = np.dot(np.linalg.inv(D),np.dot(X.transpose(),Y))
Yhat = np.dot(X,M)
f.scatter(x=x,y=Yhat,color='green')
f.line(x=x,y=Yhat,color='green')
show(f)

### Residuals

One way to evaluate the fit of the line to the data is to compare Y and Yhat.  Let's make a scatter plot of Y vs Yhat to see
how they compare.  If the fit is good, the points should cluster around the diagonal line y=x.



In [ ]:

show(comparison_plot(Y,Yhat))

In [ ]:
E = np.sum(np.square(Y-Yhat))
print('MSE={}'.format(E/Y.shape[0]))

Our MSE is 21.56.

Notice that there are a lot of cars with Yhat=30 mpg but whose true mpg (Y) are between 15 and almot 50 mpg.  This means that, while engine displacement may be a good predictor of mileage for cars with relatively low mileage, among cars with higher mileage, something else must be going on.

### Multivariate Data

For our first experiments with multivariate data, we will start with simulated data.




In [3]:
data = np.genfromtxt('../../data/multivar_simulated/data.csv',skip_header=1,delimiter=',')

Let's look at the data, just the first few rows.

In [ ]:
data[:3,:]

As the README file says, the first column (column 0) is just the row number. Column 1 is the response variable Y,
and columns 2 and 3 are the features

In [4]:
Y = data[:,1]
X = data[:,2:]

We append a column of ones to the data matrix.

Let's look at the relationship between the three columns.  First, how does the response Y depend on
the two columns of data? Plot Y vs X0 and X1 and show the result.

In [ ]:
#f0 = figure()
#f0.xaxis.axis_label=
#f0.yaxis.axis_label = 
#f0.scatter()
#f1 = figure()
#f1.xaxis.axis_label = 
#f1.yaxis.axis_label = 
#f1.scatter()
#show(gridplot())

In [ ]:
f0 = figure()
f0.xaxis.axis_label='X0'
f0.yaxis.axis_label = 'Y'
f0.scatter(x=X[:,0],y=Y)
f1 = figure()
f1.xaxis.axis_label = 'X1'
f1.yaxis.axis_label = 'Y'
f1.scatter(x=X[:,1],y=Y)
show(gridplot([[f0,f1]],plot_width=300,plot_height=300))

It appears that increasing X0 increases Y, while increasing X1 decreases Y.  What about the relationship
between X0 and X1? Plot this as well

In [ ]:
#f2 = figure()
#f2.xaxis.axis_label = 
#f2.yaxis.axis_label = 
#f2.scatter()
#show(f2)

In [ ]:
f2 = figure(width=300,height=300)
f2.xaxis.axis_label = 'X0'
f2.yaxis.axis_label = 'X1'
f2.scatter(x=X[:,0],y=X[:,1])
show(f2)

The X0 and X1 variables seem independent of each other.

Now we can do the regression.  First we add a column of 1's to our data matrix.

In [ ]:
# create a column of ones
# append it to X
# check that X has shape 75 x 3

In [5]:
E = np.ones(shape=(X.shape[0],1))
X = np.concatenate([X,E],axis=1)
X.shape

(75, 3)

Now we compute the various matrices need for the computation.

In [ ]:
#Compute the regression matrices D, Dinv, M, and Yhat

In [6]:
D = np.dot(X.transpose(),X)
Dinv = np.linalg.inv(D)
M = np.dot(np.dot(Dinv,X.transpose()),Y)
Yhat = np.dot(X,M)

Notice that the coefficients of D are large (and those of Dinv) are small:

In [ ]:
# look at D and Dinv

In [7]:
D, Dinv

(array([[ 515.72235922,  483.38478882,  156.75529436],
        [ 483.38478882, 1183.05046071,  236.7585729 ],
        [ 156.75529436,  236.7585729 ,   75.        ]]),
 array([[ 0.00532506,  0.00014004, -0.01157183],
        [ 0.00014004,  0.00229907, -0.00755038],
        [-0.01157183, -0.00755038,  0.06135417]]))

This can be a problem with accuracy, but we'll come back to it.  The components of the model that we compute are:


In [ ]:
# look at the M matrix

In [8]:
M

array([ 1.78777492, -3.47899986,  6.0608333 ])

You should have gotten a result that says that  Y is estimated as $\hat{y}=1.78 x_0 -3.47 x_1 + 6.06$  One way to visualize this is to
plot the values of Yhat vs Y to see how close they are.

In [ ]:
# plot the residuals using comparison_plot to compare Yhat and Y

In [ ]:

show(comparison_plot(Y,Yhat))

As the plot shows, the points (y,yhat) cluster around the diagonal.  

### Covariance Matrix

Let's go back to look at the covariance matrix of the variables X0 and X1.  This computed using centered coordinates.

In [ ]:
# Center the data
#X0 = (X[] - X[].mean())
# then compute D0 and Cov = D0/X0.shape[0]

In [ ]:
X0 =( X[:,:2]-X[:,:2].mean(axis=0))
D0 = np.dot(X0.transpose(),X0)
Cov = D0/X.shape[0]
Cov

Notice that the off diagonal elements, which are the covariances of X0 and X1, are small in comparison to the diagonal elements.
This reflects the fact that X0 and X1 are relatively (linearly) independent of each other.  

### The correlation coefficient

Pearson's $R^2$ is a quantitative measure of the strength of the linear relationship between two variables $X$ and $Y$. It is computed
as
$$
R^{2} = \frac{\sigma_{XY}^2}{\sigma_{X}^2\sigma_{Y}^2}
$$
From our matrix this is the offdiagonal element divided by the product of the diagonal elements.  It it's close to $1$, there is a strong relationship, if close to zero, there is not. In this case, the $R^2$ is essentially zero.

In [ ]:
R2 = Cov[0,1]**2/Cov[1,1]/Cov[0,0]
print('Correlation is {}'.format(R2))

### Multivariate MPG data

Now let's refine our look at the MPG data by considering displacement, miles per gallon, and vehicle weight.  
- column 0 is mpg
- column 2 is displacement
- column4 is vehicle weight

In [ ]:

from itertools import combinations

In [ ]:
data = np.genfromtxt('../data/auto-mpg/auto-mpg.csv',delimiter=',',skip_header=1)

In [ ]:
# this code produces a grid of plots for you to study
d={}
d['mpg'] = data[:,0]
d['displacement'] = data[:,2]
d['weight'] = data[:,4]
d['horsepower'] = data[:,3]
figs = []
for x,y in combinations(['displacement','weight','mpg'],2):
    f = figure()
    f.xaxis.axis_label = x
    f.yaxis.axis_label = y
    f.scatter(x=d[x],y=d[y])
    figs.append(f)
g = gridplot([figs[0:2],[figs[2]]],plot_height=250,plot_width=250)
show(g)

The picture above is quite different from the simulated data because all three of the combinations show a relationship (remember,
in the simulated data, the predictor variables didn't seem to be related to one another).   

Another thing to observe is that the different data are on different size scales -- vehicle weight is in the 1000's, displacmeent is in the 100's,
and mpg is in the 10's.  



In [ ]:
# build the data matrix out of the corresponding columns and the variable Y from the mpg column

In [ ]:
X = np.stack([data[:,2],data[:,4],np.ones(data.shape[0])],axis=1)
Y = data[:,0]

In [ ]:
# compute D, Dinv, M and Yhat

In [ ]:
D = np.dot(X.transpose(),X)
Dinv = np.linalg.inv(D)
M = np.dot(Dinv, np.dot(X.transpose(),Y))
Yhat = np.dot(X,M)

In [ ]:
# Look at the D matrix and its inverse and note the size of the coefficients

In [ ]:

D, Dinv

In [ ]:
# plot the residuals of Y and Yhat using comparison_plot.  What do you see?

In [ ]:
# here we compare the predicted and known values 
show(comparison_plot(Y,Yhat))

The regression coefficients are:

In [ ]:
M

In [ ]:
# compute the MSE and compare the result to the one variable computation done earlier.  Is this better?

In [ ]:
E = np.sum(np.square(Y-Yhat))
print('MSE={}'.format(E/Y.shape[0]))

Our MSE is a bit better in this multivariate case (18.4 vs 21 in the one-variable case).

Each unit increase in displacement reduces mileage by .016 and each unit increase in weight decreases mileage by -.0058.  However,
as the comparison plot shows, this  relationship continues to breakdown for cars with high mileage -- among cars that are predicted to 
have mileage around 30, the true mileage goes a lot higher, so some other factor must be at work.

### The covariance matrix and correlation

To get a closer look at the relationship between weight and displacement let's compute the covariance matrix.  To do that
we need to center the variables.

In [ ]:
# compute the covariance matrix from centered coordinates and then the correlation coefficient between weight and displacement.
# Note that they are highly correlated (R^2 is relatively close to 1).  How does this help explain why adding weight to our
# regression doesn't add much to our ability to predict mpg.

In [ ]:
X0=((X[:,:2]-X[:,:2].mean(axis=0)))
Cov = np.dot(X0.transpose(),X0)/X0.shape[0]
Cov

In [ ]:
R2 = Cov[0,1]**2/Cov[1,1]/Cov[0,0]

In [ ]:
print('Correlation is {}'.format(R2))